# **Preprocessing**

In preprocessing the data, the following steps are taken:

> * Prepare packages and setup  
> * Load in the data  
> * Tidy the data and store metadata  
> * Inspect data with various metrics  
> * Inspect data with visualisations 
> * Select locations
> * Select timeframe
> * Feature engineering
> * Perform train-validation-test-split  
> * Normalisation  
> * Create big combined normalised dataframe

### **Prepare packages and setup**

In [1]:
from pipeline import read_meteo_csv_from_data_raw
from pipeline import read_four_contaminants
from pipeline import get_metadata
from pipeline import tidy_raw_contaminant_data
from pipeline import tidy_raw_meteo_data
from pipeline import print_aggegrated_sensor_metrics
from pipeline import subset_sensors
from pipeline import perform_data_split
from pipeline import perform_data_split_without_train
from pipeline import print_split_ratios
from pipeline import calc_combined_min_max_params
from pipeline import normalise_linear
from pipeline import print_pollutant_extremes
from pipeline import export_minmax
from pipeline import plot_distributions_KDE
from pipeline import concat_frames_horizontally
from pipeline import delete_timezone_from_index
from pipeline import assert_equal_shape
from pipeline import assert_equal_index
from pipeline import assert_no_NaNs
from pipeline import assert_range


Running __init__.py for data pipeline...
Pipeline initialized



In [2]:
SUBSET_MONTHS = bool(1)                 # If true, only the months specified in the list below will be
                                        # used for the training, validation and testing set
START_MON = '08'                        # starting month for the data
END_MON = '12'                          # ending month for the data

# Days used for the training, validation and testing splits
days_vali = 21
days_test = 21

days_vali_final_yrs = 63
days_test_final_yrs = 63

# ============================================================================+

# Sensor locations in the case of Utrecht area:
DE_BILT = 'S260'                        # starting (and only used) location for meteorological data
TUINDORP = 'NL10636'                    # starting location for contamination data
BREUKELEN = 'NL10641'                   # 'goal' location for contamination data
sensors_1D = [TUINDORP, BREUKELEN]

# =============================================================================

pollutants = ['NO2'] # adjustable for 'NO2', 'PM10', 'PM25', 'O3'
selected_weather = ['T', 'TD', 'DD', 'FH', 'P', 'SQ', 'FF', 'FX'] # features selected from corr matrix, plus 'FF' and 'FX'

# Define the years for the dataset
years = [2017] # adjustable for 2017, 2018, 2020, 2021, 2022, 2023

# At multiple locations, a sys.exit() can be used to halt the script

LOG = True

### **Load in the data**

In [3]:
# Explicit variable declaration
df_NO2_2017_raw = None
df_NO2_2017_tidy = None
df_PM25_2017_raw = None
df_PM25_2017_tidy = None
df_PM10_2017_raw = None
df_PM10_2017_tidy = None
df_O3_2017_raw = None
df_O3_2017_tidy = None
df_meteo_2017_raw = None

df_NO2_2018_raw = None
df_NO2_2018_tidy = None
df_PM25_2018_raw = None
df_PM25_2018_tidy = None
df_PM10_2018_raw = None
df_PM10_2018_tidy = None
df_O3_2018_raw = None
df_O3_2018_tidy = None
df_meteo_2018_raw = None

df_NO2_2020_raw = None
df_NO2_2020_tidy = None
df_PM25_2020_raw = None
df_PM25_2020_tidy = None
df_PM10_2020_raw = None
df_PM10_2020_tidy = None
df_O3_2020_raw = None
df_O3_2020_tidy = None
df_meteo_2020_raw = None

df_NO2_2021_raw = None
df_NO2_2021_tidy = None
df_PM25_2021_raw = None
df_PM25_2021_tidy = None
df_PM10_2021_raw = None
df_PM10_2021_tidy = None
df_O3_2021_raw = None
df_O3_2021_tidy = None
df_meteo_2021_raw = None

df_NO2_2022_raw = None
df_NO2_2022_tidy = None
df_PM25_2022_raw = None
df_PM25_2022_tidy = None
df_PM10_2022_raw = None
df_PM10_2022_tidy = None
df_O3_2022_raw = None
df_O3_2022_tidy = None
df_meteo_2022_raw = None

df_NO2_2023_raw = None
df_NO2_2023_tidy = None
df_PM25_2023_raw = None
df_PM25_2023_tidy = None
df_PM10_2023_raw = None
df_PM10_2023_tidy = None
df_O3_2023_raw = None
df_O3_2023_tidy = None
df_meteo_2023_raw = None


In [4]:
# Loading raw data
for year in years:
    raw_data = read_four_contaminants(year, pollutants)
    for pollutant, data in zip(pollutants, raw_data):
        globals()[f'df_{pollutant}_{year}_raw'] = data

In [5]:
# Loading meteorological data
for year in years:
    globals()[f'df_meteo_{year}_raw'] = read_meteo_csv_from_data_raw(year)

In [6]:
df_O3_2017_raw

In [7]:
df_NO2_2017_raw

,component,matrix,meetduur,eenheid,begindatumtijd,einddatumtijd,NL01485_NO2_lucht,NL01487_NO2_lucht,NL01488_NO2_lucht,NL01489_NO2_lucht,...,NL49704_NO2_lucht,NL50002_NO2_lucht,NL50003_NO2_lucht,NL50004_NO2_lucht,NL53001_NO2_lucht,NL53004_NO2_lucht,NL53015_NO2_lucht,NL53016_NO2_lucht,NL53020_NO2_lucht,NL54004_NO2_lucht
0,NO2,lucht,uur,µg/m³,2017-01-01T00:00:00+01:00,2017-01-01T01:00:00+01:00,38.2,37.1,33.4,33.4,...,43.0,39.3,35.0,30.684,40.0,31.8,41.8,36.3,39.1,39.5
1,NO2,lucht,uur,µg/m³,2017-01-01T01:00:00+01:00,2017-01-01T02:00:00+01:00,37.8,35.9,30.8,38.8,...,51.2,33.9,28.7,25.984,35.0,30.6,40.4,34.7,37.4,40.9
2,NO2,lucht,uur,µg/m³,2017-01-01T02:00:00+01:00,2017-01-01T03:00:00+01:00,36.9,34.0,31.1,36.3,...,48.6,34.5,27.8,34.184,38.0,30.9,38.5,33.0,34.5,43.4
3,NO2,lucht,uur,µg/m³,2017-01-01T03:00:00+01:00,2017-01-01T04:00:00+01:00,36.5,32.1,28.1,31.1,...,42.0,39.9,18.4,36.285,40.0,27.4,36.2,32.4,33.0,37.0
4,NO2,lucht,uur,µg/m³,2017-01-01T04:00:00+01:00,2017-01-01T05:00:00+01:00,24.8,24.4,21.8,26.2,...,42.9,32.9,30.9,35.585,39.0,29.3,36.2,33.8,34.1,33.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,NO2,lucht,uur,µg/m³,2017-12-31T19:00:00+01:00,2017-12-31T20:00:00+01:00,9.4,22.9,15.3,19.5,...,20.7,5.1,16.9,4.300,19.0,33.1,10.0,4.7,2.4,7.1
8756,NO2,lucht,uur,µg/m³,2017-12-31T20:00:00+01:00,2017-12-31T21:00:00+01:00,7.1,13.0,12.4,17.6,...,7.4,4.2,12.7,5.100,15.0,29.1,5.8,4.3,1.7,8.5
8757,NO2,lucht,uur,µg/m³,2017-12-31T21:00:00+01:00,2017-12-31T22:00:00+01:00,6.3,11.7,11.3,12.4,...,8.9,12.3,7.5,5.600,18.0,23.0,5.7,3.4,1.9,7.6
8758,NO2,lucht,uur,µg/m³,2017-12-31T22:00:00+01:00,2017-12-31T23:00:00+01:00,5.3,8.8,8.4,9.6,...,9.9,6.7,11.2,3.700,20.0,24.5,4.4,2.7,1.7,8.1


In [8]:
df_meteo_2017_raw

,STN,YYYYMMDD,HH,DD,FH,FF,FX,T,T10N,TD,...,VV,N,U,WW,IX,M,R,S,O,Y
0,260,20170701,1,280,30,40,70,146,NaN,138,...,60,8,94,61.0,7,0,1,0,0,0
1,260,20170701,2,260,30,30,70,143,NaN,136,...,56,8,95,61.0,7,0,1,0,0,0
2,260,20170701,3,290,30,20,60,142,NaN,136,...,64,8,95,23.0,7,0,1,0,0,0
3,260,20170701,4,270,20,20,50,143,NaN,136,...,65,8,95,81.0,7,0,1,0,0,0
4,260,20170701,5,270,20,20,50,144,NaN,141,...,42,8,97,62.0,7,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4411,260,20171231,20,220,50,70,110,100,NaN,85,...,69,8,90,57.0,7,0,1,0,0,0
4412,260,20171231,21,230,70,60,120,96,NaN,80,...,62,7,89,57.0,7,0,1,0,0,0
4413,260,20171231,22,220,70,70,150,96,NaN,66,...,70,8,81,23.0,7,0,1,0,0,0
4414,260,20171231,23,220,70,60,150,89,NaN,52,...,75,6,77,NaN,5,0,0,0,0,0


In [9]:
if LOG:
    print('(1/8): Data read successfully')

(1/8): Data read successfully


### **Tidy Pollutants data**
Certain days which might affect pollutants are removed

Also only made it start from August (START_MON) until December (END_MON)

In [10]:
# Metadata extraction
for year in years:
    for pollutant in pollutants:
        raw_data_var = f'df_{pollutant}_{year}_raw'
        meta_var = f'{pollutant}_{year}_meta'
        if raw_data_var in globals():
            globals()[meta_var] = get_metadata(globals()[raw_data_var])
        else:
            print(f"Warning: {raw_data_var} is not defined.")

# Tidying data
for year in years:
    for pollutant in pollutants:
        raw_data_var = f'df_{pollutant}_{year}_raw'
        tidy_var = f'df_{pollutant}_{year}_tidy'
        if raw_data_var in globals():
            globals()[tidy_var] = tidy_raw_contaminant_data(
                globals()[raw_data_var], str(year), SUBSET_MONTHS, START_MON, END_MON
            )
        else:
            print(f"Warning: {raw_data_var} is not defined.")


In [11]:
df_NO2_2017_raw

,component,matrix,meetduur,eenheid,begindatumtijd,einddatumtijd,NL01485_NO2_lucht,NL01487_NO2_lucht,NL01488_NO2_lucht,NL01489_NO2_lucht,...,NL49704_NO2_lucht,NL50002_NO2_lucht,NL50003_NO2_lucht,NL50004_NO2_lucht,NL53001_NO2_lucht,NL53004_NO2_lucht,NL53015_NO2_lucht,NL53016_NO2_lucht,NL53020_NO2_lucht,NL54004_NO2_lucht
0,NO2,lucht,uur,µg/m³,2017-01-01T00:00:00+01:00,2017-01-01T01:00:00+01:00,38.2,37.1,33.4,33.4,...,43.0,39.3,35.0,30.684,40.0,31.8,41.8,36.3,39.1,39.5
1,NO2,lucht,uur,µg/m³,2017-01-01T01:00:00+01:00,2017-01-01T02:00:00+01:00,37.8,35.9,30.8,38.8,...,51.2,33.9,28.7,25.984,35.0,30.6,40.4,34.7,37.4,40.9
2,NO2,lucht,uur,µg/m³,2017-01-01T02:00:00+01:00,2017-01-01T03:00:00+01:00,36.9,34.0,31.1,36.3,...,48.6,34.5,27.8,34.184,38.0,30.9,38.5,33.0,34.5,43.4
3,NO2,lucht,uur,µg/m³,2017-01-01T03:00:00+01:00,2017-01-01T04:00:00+01:00,36.5,32.1,28.1,31.1,...,42.0,39.9,18.4,36.285,40.0,27.4,36.2,32.4,33.0,37.0
4,NO2,lucht,uur,µg/m³,2017-01-01T04:00:00+01:00,2017-01-01T05:00:00+01:00,24.8,24.4,21.8,26.2,...,42.9,32.9,30.9,35.585,39.0,29.3,36.2,33.8,34.1,33.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8755,NO2,lucht,uur,µg/m³,2017-12-31T19:00:00+01:00,2017-12-31T20:00:00+01:00,9.4,22.9,15.3,19.5,...,20.7,5.1,16.9,4.300,19.0,33.1,10.0,4.7,2.4,7.1
8756,NO2,lucht,uur,µg/m³,2017-12-31T20:00:00+01:00,2017-12-31T21:00:00+01:00,7.1,13.0,12.4,17.6,...,7.4,4.2,12.7,5.100,15.0,29.1,5.8,4.3,1.7,8.5
8757,NO2,lucht,uur,µg/m³,2017-12-31T21:00:00+01:00,2017-12-31T22:00:00+01:00,6.3,11.7,11.3,12.4,...,8.9,12.3,7.5,5.600,18.0,23.0,5.7,3.4,1.9,7.6
8758,NO2,lucht,uur,µg/m³,2017-12-31T22:00:00+01:00,2017-12-31T23:00:00+01:00,5.3,8.8,8.4,9.6,...,9.9,6.7,11.2,3.700,20.0,24.5,4.4,2.7,1.7,8.1


In [12]:
df_NO2_2017_tidy

,NL01485,NL01487,NL01488,NL01489,NL01491,NL01493,NL01494,NL01495,NL01496,NL01908,...,NL49703,NL49704,NL50002,NL50003,NL53001,NL53004,NL53015,NL53016,NL53020,NL54004
DateTime,,,,,,,,,,,,,,,,,,,,,
2017-08-01 00:00:00,65.7,57.1,46.0,33.4,51.0,66.3,66.3,55.0,39.3,24.8,...,20.6,56.1,22.8,17.0,11.0,22.5,22.5,28.7,34.8,29.0
2017-08-01 01:00:00,59.6,53.1,46.2,25.0,47.9,64.4,67.6,34.6,46.8,16.4,...,20.6,48.1,21.3,16.8,11.0,18.8,35.2,31.4,40.1,21.1
2017-08-01 02:00:00,53.3,47.8,34.6,32.5,47.4,52.3,58.1,29.2,19.9,26.2,...,23.1,52.2,21.3,18.3,12.0,17.1,34.6,22.6,31.3,22.4
2017-08-01 03:00:00,45.8,48.5,45.1,41.4,49.3,49.3,62.5,38.6,17.2,32.9,...,28.3,41.8,28.1,27.5,10.0,12.1,23.5,19.4,25.8,27.7
2017-08-01 04:00:00,31.1,52.1,49.3,44.3,48.5,59.6,59.0,51.2,37.2,49.9,...,30.4,33.5,21.8,19.4,19.0,17.9,25.8,30.6,19.7,40.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-30 19:00:00,9.4,19.7,15.9,23.9,23.9,38.8,21.8,25.2,43.4,11.7,...,9.3,21.5,19.1,18.4,9.0,33.4,8.2,9.4,0.8,18.7
2017-12-30 20:00:00,11.3,19.7,15.1,21.4,26.7,41.8,22.9,19.9,35.7,9.7,...,10.2,18.8,7.7,12.0,8.0,35.7,7.9,9.6,1.1,19.4
2017-12-30 21:00:00,9.4,24.8,21.8,16.4,21.2,25.2,22.5,18.3,37.6,8.2,...,10.2,18.5,5.5,13.9,6.0,30.2,10.2,9.3,2.2,17.6


In [13]:
pollutant_data = []
for year in years:
    for pollutant in pollutants:
        pollutant_data.append(eval(f'df_{pollutant}_{year}_tidy'))

assert_equal_shape(pollutant_data, True, False, 'Tidying of pollutant data')
print('(2/8): Pollutant data tidied successfully')

(2/8): Pollutant data tidied successfully


In [14]:
df_NO2_2017_tidy

,NL01485,NL01487,NL01488,NL01489,NL01491,NL01493,NL01494,NL01495,NL01496,NL01908,...,NL49703,NL49704,NL50002,NL50003,NL53001,NL53004,NL53015,NL53016,NL53020,NL54004
DateTime,,,,,,,,,,,,,,,,,,,,,
2017-08-01 00:00:00,65.7,57.1,46.0,33.4,51.0,66.3,66.3,55.0,39.3,24.8,...,20.6,56.1,22.8,17.0,11.0,22.5,22.5,28.7,34.8,29.0
2017-08-01 01:00:00,59.6,53.1,46.2,25.0,47.9,64.4,67.6,34.6,46.8,16.4,...,20.6,48.1,21.3,16.8,11.0,18.8,35.2,31.4,40.1,21.1
2017-08-01 02:00:00,53.3,47.8,34.6,32.5,47.4,52.3,58.1,29.2,19.9,26.2,...,23.1,52.2,21.3,18.3,12.0,17.1,34.6,22.6,31.3,22.4
2017-08-01 03:00:00,45.8,48.5,45.1,41.4,49.3,49.3,62.5,38.6,17.2,32.9,...,28.3,41.8,28.1,27.5,10.0,12.1,23.5,19.4,25.8,27.7
2017-08-01 04:00:00,31.1,52.1,49.3,44.3,48.5,59.6,59.0,51.2,37.2,49.9,...,30.4,33.5,21.8,19.4,19.0,17.9,25.8,30.6,19.7,40.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-12-30 19:00:00,9.4,19.7,15.9,23.9,23.9,38.8,21.8,25.2,43.4,11.7,...,9.3,21.5,19.1,18.4,9.0,33.4,8.2,9.4,0.8,18.7
2017-12-30 20:00:00,11.3,19.7,15.1,21.4,26.7,41.8,22.9,19.9,35.7,9.7,...,10.2,18.8,7.7,12.0,8.0,35.7,7.9,9.6,1.1,19.4
2017-12-30 21:00:00,9.4,24.8,21.8,16.4,21.2,25.2,22.5,18.3,37.6,8.2,...,10.2,18.5,5.5,13.9,6.0,30.2,10.2,9.3,2.2,17.6


In [15]:
df_meteo_2017_raw.columns

Index(['STN', 'YYYYMMDD', 'HH', 'DD', 'FH', 'FF', 'FX', 'T', 'T10N', 'TD',
       'SQ', 'Q', 'DR', 'RH', 'P', 'VV', 'N', 'U', 'WW', 'IX', 'M', 'R', 'S',
       'O', 'Y'],
      dtype='object')

In [16]:
# print the column of 2017 meteo data from 'FF'
df_meteo_2017_raw['FF']

0       40
1       30
2       20
3       20
4       20
        ..
4411    70
4412    60
4413    70
4414    60
4415    90
Name: FF, Length: 4416, dtype: int64

## **Extract and tidy meteorological data**

In [17]:
only_DeBilt = True  # True: only De Bilt is used
dataframes = {}

for year in years:
    raw_data = globals()[f'df_meteo_{year}_raw']
    for var in selected_weather:
        var_name = var # if var != 'P' else 'P_'  # Handling 'P' separately to match original naming (removed for now)
        df_name = f"df_{var_name}_{year}_tidy"
        dataframes[df_name] = tidy_raw_meteo_data(
            raw_data, var, only_DeBilt, str(year), SUBSET_MONTHS, START_MON, END_MON)

# Assign variables dynamically
globals().update(dataframes)


In [18]:
# print all the df names of dataframes
for df_name in dataframes:
    print(df_name)


df_T_2017_tidy
df_TD_2017_tidy
df_DD_2017_tidy
df_FH_2017_tidy
df_P_2017_tidy
df_SQ_2017_tidy
df_FF_2017_tidy
df_FX_2017_tidy


In [19]:
df_T_2017_tidy

,S260
DateTime,
2017-08-01 00:00:00,149.0
2017-08-01 01:00:00,152.0
2017-08-01 02:00:00,140.0
2017-08-01 03:00:00,127.0
2017-08-01 04:00:00,136.0
...,...
2017-12-30 19:00:00,111.0
2017-12-30 20:00:00,108.0
2017-12-30 21:00:00,103.0


In [20]:
# Extract all DataFrames from the dictionary
datasets = list(dataframes.values())

# Ensure there is at least one dataset to check
assert len(datasets) > 0, "Error: No datasets found!"

# 1. Assert all datasets have the same shape
reference_shape = datasets[0].shape  # Take the shape of the first dataset as reference
assert all(df.shape == reference_shape for df in datasets), "Error: Not all datasets have the same shape!"

# 2. Assert no NaNs in any dataset
assert all(not df.isnull().values.any() for df in datasets), "Error: Some datasets contain NaN values!"

print("All datasets have the same shape and contain no NaN values.")
print('(3/8): Meteorological data tidied successfully')


All datasets have the same shape and contain no NaN values.
(3/8): Meteorological data tidied successfully


### **Inspect data with various metrics**

pipeline/statistics.py contains more functions

In [21]:
print("Printing some basic statistics for the pollutants:")
print("(Sensor NL10636 is TUINDORP)\n")

for pollutant in pollutants:
    dataframes = []
    for year in years:
        df_name = f'df_{pollutant}_{year}_tidy'
        try:
            dataframes.append(eval(df_name))
        except NameError:
            print(f"Warning: {df_name} does not exist")
    
    if dataframes:
        meta_name = f'{pollutant}_2017_meta' # meta data is just to know the pollutant name and unit
        print_aggegrated_sensor_metrics(dataframes, TUINDORP, eval(meta_name))


Printing some basic statistics for the pollutants:
(Sensor NL10636 is TUINDORP)

[min, mean, max] for sensor NL10636 measuring NO2 µg/m³
aggregated over multiple years:
[1.3700, 29.9679, 93.0900] with n = 3648



In [22]:
# Delete pollutant data
for year in years:
    for pollutant in pollutants:
        var_name = f"df_{pollutant}_{year}_raw"
        if var_name in globals():
            del globals()[var_name]

# Delete meteorological data
for year in years:
    var_name = f"df_meteo_{year}_raw"
    if var_name in globals():
        del globals()[var_name]


### **Inspect data with visualisations**

plots.py contains plotting functions

### **Select locations**

In [23]:
# Here, we'll select the locations we want to use. The
# generated dataframes will be 1-dimensional
# datasets are called df_pollutant_year_tidy_subset_1D


# Create subset dataframes dynamically
for year in years:
    for pollutant in pollutants:
        raw_var = f"df_{pollutant}_{year}_tidy"
        subset_var = f"{raw_var}_subset_1D"
        
        if raw_var in globals():
            globals()[subset_var] = subset_sensors(globals()[raw_var], sensors_1D)

# Delete original tidy dataframes after subsetting
for year in years:
    for pollutant in pollutants:
        raw_var = f"df_{pollutant}_{year}_tidy"
        if raw_var in globals():
            del globals()[raw_var]


In [24]:
df_NO2_2017_tidy_subset_1D

,NL10636,NL10641
DateTime,,
2017-08-01 00:00:00,23.785,22.08
2017-08-01 01:00:00,22.030,14.84
2017-08-01 02:00:00,16.730,26.92
2017-08-01 03:00:00,14.550,40.60
2017-08-01 04:00:00,15.840,45.69
...,...,...
2017-12-30 19:00:00,17.910,21.44
2017-12-30 20:00:00,14.810,23.19
2017-12-30 21:00:00,20.300,23.58


Making sure all years and pollutants datasets have the same shapes

In [25]:
for year in years:
    for pollutant in pollutants:
        df_name = f'df_{pollutant}_{year}_tidy_subset_1D'
        try:
            df = eval(df_name)
            print(f"Year: {year}, Pollutant: {pollutant}, Shape: {df.shape}")
        except NameError:
            print(f"Warning: {df_name} does not exist")

Year: 2017, Pollutant: NO2, Shape: (3648, 2)


### **Perform assertions for pollutant tidy 1D data**

In [26]:
if LOG:
    
    # Collect dataframes dynamically
    shape_check_dfs = [
        globals()[f"df_{pollutant}_{year}_tidy_subset_1D"]
        for year in years
            for pollutant in pollutants
                if f"df_{pollutant}_{year}_tidy_subset_1D" in globals()
    ]

    nan_check_dfs = [
        globals()[f"df_{pollutant}_{year}_tidy_subset_1D"]
        for year in years
            for pollutant in pollutants
                if f"df_{pollutant}_{year}_tidy_subset_1D" in globals()
    ]

    # Perform assertions
    assert_equal_shape(shape_check_dfs, True, True, 'Location-wise subsetting of pollutant data')
    assert_no_NaNs(nan_check_dfs, 'Location-wise subsetting of pollutant data')

    print('(4/8): Location-wise subsetting of pollutant data successful')


(4/8): Location-wise subsetting of pollutant data successful


### **Select timeframe**

Timeframe selection (excluding 2016 and 2019) was done iteratively and manually by inspecting the data and inspecting the distributions. It is discussed in the thesis, Section 3.1, from Valentijn's thesis.

This has already been done in the beginning of the script.


### **Feature Engineering**

This is discussed in Section 3.2 of the thesis and done mainly with a plain correlation matrix.
I'll use the same features that Valentijn has selected from correlation matrix, but add other wind features as well for the PEML model.

### **Finding number of days for validation and test to achirve 75-15-15 splits**

In [ ]:
import pandas as pd
# Calculate the total number of days from August 1st to December 30th
# if years are not [2017, 2018, 2020, 2021, 2022, 2023], then days vali and test are calculated manually
# if years are [2017, 2018, 2020, 2021, 2022, 2023], then days vali and test are already defined at the top
if years != [2017, 2018, 2020, 2021, 2022, 2023]:
    print("Assuming start date is August 1st and end date is December 30th in 2017.")
    print("Change accordingly if that's not the case")
    start_date = pd.Timestamp('2017-08-01')
    end_date = pd.Timestamp('2017-12-30')
    total_days = (end_date - start_date).days + 1

    # Calculate 15% of the total number of days for validation and test sets
    days_vali = int(total_days * 0.15)
    days_test = int(total_days * 0.15)
    print(days_vali, days_test)

Assuming start date is August 1st and end date is December 30th in 2017.
change accordingly if that's not the case
22 22


### **Perform train-validation-test-split**

In [28]:
# Splitting the data according to the years
# According to the splitting defined from Valentijn, 2017 2018 2020 are used for training
# 2021 2022 are used for validation and 2023 is used for testing
if len(years) == 1:
    year = years[0]
    years_splitted = years
    years_unsplitted = []
    year_final = 2023 # if only one year is used, the same year is used for training, validation and testing
    for pollutant in pollutants:
        df_train, df_val, df_test = perform_data_split(eval(f'df_{pollutant}_{year}_tidy_subset_1D'), days_vali, days_test)
        exec(f'df_{pollutant}_{year}_train_1D = df_train')
        exec(f'df_{pollutant}_{year}_val_1D = df_val')
        exec(f'df_{pollutant}_{year}_test_1D = df_test')

    for meteo in selected_weather:
        df_train, df_val, df_test = perform_data_split(eval(f'df_{meteo}_{year}_tidy'), days_vali, days_test)
        exec(f'df_{meteo}_{year}_train = df_train')
        exec(f'df_{meteo}_{year}_val = df_val')
        exec(f'df_{meteo}_{year}_test = df_test')
elif years == [2017, 2018, 2020, 2021, 2022, 2023]:
    years_unsplitted = [2017, 2018, 2020]
    years_splitted = [2021, 2022]
    year_final = 2023
    for year in years:
        if year in years_unsplitted:
            for pollutant in pollutants:
                exec(f'df_{pollutant}_{year}_train_1D = df_{pollutant}_{year}_tidy_subset_1D.copy()')
            for meteo in selected_weather:
                exec(f'df_{meteo}_{year}_train = df_{meteo}_{year}_tidy.copy()')
        elif year in years_splitted:
            for pollutant in pollutants:
                exec(f"""
                    df_{pollutant}_{year}_train_1D, df_{pollutant}_{year}_val_1D, df_{pollutant}_{year}_test_1D = 
                    perform_data_split(df_{pollutant}_{year}_tidy_subset_1D, days_vali, days_test)
                                """)
            for meteo in selected_weather:
                exec(f"""
                df_{meteo}_{year}_train, df_{meteo}_{year}_val, df_{meteo}_{year}_test = 
                perform_data_split(df_{meteo}_{year}_tidy, days_vali, days_test)
                            """)
        elif year == year_final:
            for pollutant in pollutants:
                exec(f"""
                df_{pollutant}_{year}_val_1D, df_{pollutant}_{year}_test_1D = 
                perform_data_split_without_train(df_{pollutant}_{year}_tidy_subset_1D, days_vali_final_yrs, days_test_final_yrs)
                            """)
            for meteo in selected_weather:
                exec(f"""
                df_{meteo}_{year}_val, df_{meteo}_{year}_test = 
                perform_data_split_without_train(df_{meteo}_{year}_tidy, days_vali_final_yrs, days_test_final_yrs)
                            """)
else:
    print("Error: train val test split not defined for the years")

### Checking if shapes are the same for all datasets

In [29]:
if LOG:
     # First, check for equal shape of pollutant data of unsplitted years
    pollutant_data_unsplitted = []
    for year in years_unsplitted:
        if year in years:
            for pollutant in pollutants:
                df_name = f'df_{pollutant}_{year}_train_1D'
                pollutant_data_unsplitted.append(eval(df_name))
    assert_equal_shape(pollutant_data_unsplitted, True, True, 'Split of pollutant train set for 2017, 2018 and 2020')

    # Second, check for equal shape of meteorological data of unsplitted years
    meteorological_data_unsplitted = []
    for year in years_unsplitted:
        if year in years:
            for meteo in selected_weather:
                df_name = f'df_{meteo}_{year}_train'
                meteorological_data_unsplitted.append(eval(df_name))
    assert_equal_shape(meteorological_data_unsplitted, True, True, 'Split of meteorological train set for 2017, 2018 and 2020')

    # Third, check for equal row number of training set in 2021 and 2022
    training_data_splitted = []
    for year in years_splitted:
        if year in years:
            for pollutant in pollutants:
                df_name = f'df_{pollutant}_{year}_train_1D'
                training_data_splitted.append(eval(df_name))
            for meteo in selected_weather:
                df_name = f'df_{meteo}_{year}_train'
                training_data_splitted.append(eval(df_name))
    assert_equal_shape(training_data_splitted, True, False, 'Split of training data for 2021 and 2022')

    # Fourth, check for equal row number of validation set in 2021 and 2022
    validation_data_splitted = []
    for year in years_splitted:
        if year in years:
            for pollutant in pollutants:
                df_name = f'df_{pollutant}_{year}_val_1D'
                validation_data_splitted.append(eval(df_name))
            for meteo in selected_weather:
                df_name = f'df_{meteo}_{year}_val'
                validation_data_splitted.append(eval(df_name))
    assert_equal_shape(validation_data_splitted, True, False, 'Split of validation data for 2021 and 2022')

    # Fifth, check for equal row number of test set in 2021 and 2022
    test_data_splitted = []
    for year in years_splitted:
        if year in years:
            for pollutant in pollutants:
                df_name = f'df_{pollutant}_{year}_test_1D'
                test_data_splitted.append(eval(df_name))
            for meteo in selected_weather:
                df_name = f'df_{meteo}_{year}_test'
                test_data_splitted.append(eval(df_name))
    assert_equal_shape(test_data_splitted, True, False, 'Split of test data for 2021 and 2022')

    # Sixth, check for equal row number of validation set in 2023
    validation_data_final = []
    if year_final in years:
        for pollutant in pollutants:
            df_name = f'df_{pollutant}_{year_final}_val_1D'
            validation_data_final.append(eval(df_name))
        for meteo in selected_weather:
            df_name = f'df_{meteo}_{year_final}_val'
            validation_data_final.append(eval(df_name))
    assert_equal_shape(validation_data_final, True, False, 'Split of validation data for 2023')

    # Seventh, check for equal row number of test set in 2023
    test_data_final = []
    if year_final in years:
        for pollutant in pollutants:
            df_name = f'df_{pollutant}_{year_final}_test_1D'
            test_data_final.append(eval(df_name))
        for meteo in selected_weather:
            df_name = f'df_{meteo}_{year_final}_test'
            test_data_final.append(eval(df_name))
    assert_equal_shape(test_data_final, True, False, 'Split of test data for 2023')

    print('(5/8): Train-validation-test split successful')
    del pollutant_data_unsplitted, meteorological_data_unsplitted, training_data_splitted, validation_data_splitted, test_data_splitted, validation_data_final, test_data_final

(5/8): Train-validation-test split successful


In [30]:
df_NO2_2017_test_1D

,NL10636,NL10641
DateTime,,
2017-12-09 00:00:00,12.04,11.26
2017-12-09 01:00:00,15.84,8.93
2017-12-09 02:00:00,10.45,6.87
2017-12-09 03:00:00,9.13,5.85
2017-12-09 04:00:00,18.47,7.48
...,...,...
2017-12-30 19:00:00,17.91,21.44
2017-12-30 20:00:00,14.81,23.19
2017-12-30 21:00:00,20.30,23.58


### Double checking the train val test split ratio

In [31]:
for pollutant in pollutants:
    train_dfs = []
    val_dfs = []
    test_dfs = []

    for year in years_unsplitted:
        if year in years:
            df_name = f'df_{pollutant}_{year}_train_1D'
            train_dfs.append(eval(df_name))

    for year in years_splitted:
        if year in years:
            train_dfs.append(eval(f'df_{pollutant}_{year}_train_1D'))
            val_dfs.append(eval(f'df_{pollutant}_{year}_val_1D'))
            test_dfs.append(eval(f'df_{pollutant}_{year}_test_1D'))

    if year_final in years:
        val_dfs.append(eval(f'df_{pollutant}_{year_final}_val_1D'))
        test_dfs.append(eval(f'df_{pollutant}_{year_final}_test_1D'))

    print_split_ratios(train_dfs, val_dfs, test_dfs, pollutant)
del train_dfs, val_dfs, test_dfs

[train/validation/test] %-ratio for NO2 data is: [71.1/14.5/14.5]


### **Normalisation**

Warning: make sure the pollutants are parsed in, in order of NO2, O3, PM2.5, PM10

#### Finding Min Max for pollutants and weather data

In [32]:
# Normalise each component separately, using the training data extremes


# Calculate min and max parameters for pollutants
pollutant_min_max = {}
for pollutant in pollutants:
    dfs = [eval(f'df_{pollutant}_{year}_train_1D') for year in years]
    min_train, max_train = calc_combined_min_max_params(dfs)
    pollutant_min_max[f'{pollutant}_min_train'] = min_train
    pollutant_min_max[f'{pollutant}_max_train'] = max_train

# Calculate min and max parameters for meteorological data
weather_min_max = {}
for meteo in selected_weather:
    dfs = [eval(f'df_{meteo}_{year}_train') for year in years]
    min_train, max_train = calc_combined_min_max_params(dfs)
    weather_min_max[f'{meteo}_min_train'] = min_train
    weather_min_max[f'{meteo}_max_train'] = max_train

# Print pollutant extremes
print()
df_minmax = print_pollutant_extremes(
    [pollutant_min_max[f'{pollutant}_min_train'] for pollutant in pollutants] +
    [pollutant_min_max[f'{pollutant}_max_train'] for pollutant in pollutants],
    pollutants
)
print()

# Export min and max parameters
export_minmax(df_minmax, 'pollutants_minmax')


      min    max
NO2  1.37  93.95



### Normalising according to min max

In [33]:
# Normalise each component separately, using the training data extremes

print("------Normalising the pollutants------")
for pollutant in pollutants:
    print(f"Normalising {pollutant}...")
    min_train = pollutant_min_max[f'{pollutant}_min_train']
    max_train = pollutant_min_max[f'{pollutant}_max_train']
    for year in years:
        for split in ['train_1D', 'val_1D', 'test_1D']:
            df_name = f'df_{pollutant}_{year}_{split}'
            norm_df_name = f'df_{pollutant}_{year}_{split}_norm'
            try:
                df = eval(df_name)
                norm_df = normalise_linear(df, min_train, max_train)
                globals()[norm_df_name] = norm_df
                print(f"Normalised {df_name}")
            except NameError:
                print(f"Warning: {df_name} does not exist.")

print("-------Normalising the weather data-------")
for meteo in selected_weather:
    print(f"Normalising {meteo}...")
    min_train = weather_min_max[f'{meteo}_min_train']
    max_train = weather_min_max[f'{meteo}_max_train']
    for year in years:
        for split in ['train', 'val', 'test']:
            df_name = f'df_{meteo}_{year}_{split}'
            norm_df_name = f'df_{meteo}_{year}_{split}_norm'
            try:
                df = eval(df_name)
                norm_df = normalise_linear(df, min_train, max_train)
                globals()[norm_df_name] = norm_df
                print(f"Normalised {df_name}")
            except NameError:
                print(f"Warning: {df_name} does not exist.")

------Normalising the pollutants------
Normalising NO2...
Normalised df_NO2_2017_train_1D
Normalised df_NO2_2017_val_1D
Normalised df_NO2_2017_test_1D
-------Normalising the weather data-------
Normalising T...
Normalised df_T_2017_train
Normalised df_T_2017_val
Normalised df_T_2017_test
Normalising TD...
Normalised df_TD_2017_train
Normalised df_TD_2017_val
Normalised df_TD_2017_test
Normalising DD...
Normalised df_DD_2017_train
Normalised df_DD_2017_val
Normalised df_DD_2017_test
Normalising FH...
Normalised df_FH_2017_train
Normalised df_FH_2017_val
Normalised df_FH_2017_test
Normalising P...
Normalised df_P_2017_train
Normalised df_P_2017_val
Normalised df_P_2017_test
Normalising SQ...
Normalised df_SQ_2017_train
Normalised df_SQ_2017_val
Normalised df_SQ_2017_test
Normalising FF...
Normalised df_FF_2017_train
Normalised df_FF_2017_val
Normalised df_FF_2017_test
Normalising FX...
Normalised df_FX_2017_train
Normalised df_FX_2017_val
Normalised df_FX_2017_test


In [34]:
if LOG:
    # Assert range only for training frames, validation and test
    # frames can, very theoretically, have unlimited values
    for pollutant in pollutants:
        train_dfs = []
        for year in years:
            df_name = f'df_{pollutant}_{year}_train_norm_1D'
            try:
                df = eval(df_name)
                train_dfs.append(df)
            except NameError:
                pass
        assert_range(train_dfs, 0, 1, f'Normalisation of {pollutant} data')

    for meteo in selected_weather:
        train_dfs = []
        for year in years:
            df_name = f'df_{meteo}_{year}_train_norm'
            try:
                df = eval(df_name)
                train_dfs.append(df)
            except NameError:
                pass
        assert_range(train_dfs, 0, 1, f'Normalisation of {meteo} data')

    print('(6/8): Normalisation successful')

(6/8): Normalisation successful


In [35]:
df_NO2_2017_train_1D_norm

,NL10636,NL10641
DateTime,,
2017-08-01 00:00:00,0.242115,0.223698
2017-08-01 01:00:00,0.223158,0.145496
2017-08-01 02:00:00,0.165911,0.275978
2017-08-01 03:00:00,0.142363,0.423742
2017-08-01 04:00:00,0.156297,0.478721
...,...,...
2017-11-16 19:00:00,0.523871,0.606502
2017-11-16 20:00:00,0.512314,0.456470
2017-11-16 21:00:00,0.232880,0.483258


### **Create big combined normalised dataframe**

#### Creating all the input dataframes with data from Tuindorp

In [42]:
# Create a dictionary to store the frames for each year and split
frames_dict_u = {}

splits = ['train', 'val', 'test']

# Loop through each year and split to create the combined normalized dataframes
for year in years:
    frames_dict_u[year] = {}
    for split in splits:
        frames_dict_u[year][split] = {}
        for pollutant in pollutants:
            df_name = f'df_{pollutant}_{year}_{split}_1D_norm'
            try:
                df = eval(df_name)
                frames_dict_u[year][split][pollutant] = df.loc[:, [TUINDORP]]
                print(f"frame dict added {pollutant}_{split}_{year}")
            except NameError:
                print(f"Warning: {df_name} does not exist.")
        for meteo in selected_weather:
            df_name = f'df_{meteo}_{year}_{split}_norm'
            try:
                df = eval(df_name)
                frames_dict_u[year][split][meteo] = df
            except NameError:
                print(f"Warning: {df_name} does not exist.")

# Print all the names of frames in frames_dict_u
print("Frames in frames_dict_u:")
for year in frames_dict_u:
    for split in frames_dict_u[year]:
        for key in frames_dict_u[year][split]:
            print(f"{key}_{split}_{year}")

# Example of accessing specific frames
print(frames_dict_u[2017]['train']['NO2'])
print("---------------")
print(frames_dict_u[2017]['val']['T'])

frame dict added NO2_train_2017
frame dict added NO2_val_2017
frame dict added NO2_test_2017
Frames in frames_dict_u:
NO2_train_2017
T_train_2017
TD_train_2017
DD_train_2017
FH_train_2017
P_train_2017
SQ_train_2017
FF_train_2017
FX_train_2017
NO2_val_2017
T_val_2017
TD_val_2017
DD_val_2017
FH_val_2017
P_val_2017
SQ_val_2017
FF_val_2017
FX_val_2017
NO2_test_2017
T_test_2017
TD_test_2017
DD_test_2017
FH_test_2017
P_test_2017
SQ_test_2017
FF_test_2017
FX_test_2017
                      NL10636
DateTime                     
2017-08-01 00:00:00  0.242115
2017-08-01 01:00:00  0.223158
2017-08-01 02:00:00  0.165911
2017-08-01 03:00:00  0.142363
2017-08-01 04:00:00  0.156297
...                       ...
2017-11-16 19:00:00  0.523871
2017-11-16 20:00:00  0.512314
2017-11-16 21:00:00  0.232880
2017-11-16 22:00:00  0.108123
2017-11-16 23:00:00  0.205120

[2592 rows x 1 columns]
---------------
                         S260
DateTime                     
2017-11-17 00:00:00  0.186667
2017-11-17 01

#### Creating all the input dataframes with data from Breukelen

In [44]:
# Create a dictionary to store the frames for each year and split
frames_dict_y = {}

# Loop through each year and split to create the combined normalized dataframes for Breukelen
for year in years:
    frames_dict_y[year] = {}
    for split in splits:
        frames_dict_y[year][split] = {}
        for pollutant in pollutants:
            df_name = f'df_{pollutant}_{year}_{split}_1D_norm'
            try:
                df = eval(df_name)
                frames_dict_y[year][split][pollutant] = df.loc[:, [BREUKELEN]]
                print(f"frame dict added {pollutant}_{split}_{year} for Breukelen")
            except NameError:
                print(f"Warning: {df_name} does not exist.")
# Print all the names of frames in frames_dict_y
print("Frames in frames_dict_y:")
for year in frames_dict_y:
    for split in frames_dict_y[year]:
        for key in frames_dict_y[year][split]:
            print(f"{key}_{split}_{year}")

# Example of accessing specific frames
print(frames_dict_y[2017]['train']['NO2'])
print(frames_dict_y[2017]['val']['NO2'])



frame dict added NO2_train_2017 for Breukelen
frame dict added NO2_val_2017 for Breukelen
frame dict added NO2_test_2017 for Breukelen
Frames in frames_dict_y:
NO2_train_2017
NO2_val_2017
NO2_test_2017
                      NL10641
DateTime                     
2017-08-01 00:00:00  0.223698
2017-08-01 01:00:00  0.145496
2017-08-01 02:00:00  0.275978
2017-08-01 03:00:00  0.423742
2017-08-01 04:00:00  0.478721
...                       ...
2017-11-16 19:00:00  0.606502
2017-11-16 20:00:00  0.456470
2017-11-16 21:00:00  0.483258
2017-11-16 22:00:00  0.468784
2017-11-16 23:00:00  0.473428

[2592 rows x 1 columns]
                      NL10641
DateTime                     
2017-11-17 00:00:00  0.364442
2017-11-17 01:00:00  0.331821
2017-11-17 02:00:00  0.367898
2017-11-17 03:00:00  0.460899
2017-11-17 04:00:00  0.494275
...                       ...
2017-12-08 19:00:00  0.246057
2017-12-08 20:00:00  0.237740
2017-12-08 21:00:00  0.240117
2017-12-08 22:00:00  0.297040
2017-12-08 23:00:00  0.

In [45]:
input_keys = pollutants + selected_weather
output_keys = pollutants

In [46]:
input_keys, output_keys

(['NO2', 'T', 'TD', 'DD', 'FH', 'P', 'SQ', 'FF', 'FX'], ['NO2'])

### Concatenating and putting the keys in alpha order

In [48]:
# Create dictionaries to store the concatenated dataframes for each year and split
concat_frames_dict_u = {}
concat_frames_dict_y = {}

# Loop through each year and split to concatenate the frames horizontally for Tuindorp
for year in years:
    concat_frames_dict_u[year] = {}
    for split in splits:
        frames = []
        for key in input_keys:
            try:
                frames.append(frames_dict_u[year][split][key])
            except KeyError:
                print(f"Warning: {key}_{split}_{year} does not exist.")
        concat_frames_dict_u[year][split] = concat_frames_horizontally(frames, input_keys)
        print(f"Concatenated Input frames for {split} {year} for Tuindorp")

# Loop through each year and split to concatenate the frames horizontally for Breukelen
for year in years:
    concat_frames_dict_y[year] = {}
    for split in splits:
        frames = []
        for key in output_keys:
            try:
                frames.append(frames_dict_y[year][split][key])
            except KeyError:
                print(f"Warning: {key}_{split}_{year} does not exist.")
        concat_frames_dict_y[year][split] = concat_frames_horizontally(frames, output_keys)
        print(f"Concatenated Output frames for {split} {year} for Breukelen")

# Example of accessing specific concatenated frames
print(concat_frames_dict_u[2017]['train'])
print(concat_frames_dict_y[2017]['train'])

Concatenated Input frames for train 2017 for Tuindorp
Concatenated Input frames for val 2017 for Tuindorp
Concatenated Input frames for test 2017 for Tuindorp
Concatenated Output frames for train 2017 for Breukelen
Concatenated Output frames for val 2017 for Breukelen
Concatenated Output frames for test 2017 for Breukelen
                           DD   FF        FH        FX       NO2         P  \
DateTime                                                                     
2017-08-01 00:00:00  0.166667  0.1  0.111111  0.000000  0.242115  0.562982   
2017-08-01 01:00:00  0.000000  0.0  0.111111  0.052632  0.223158  0.570694   
2017-08-01 02:00:00  0.000000  0.0  0.000000  0.000000  0.165911  0.560411   
2017-08-01 03:00:00  0.277778  0.1  0.000000  0.000000  0.142363  0.555270   
2017-08-01 04:00:00  0.805556  0.2  0.111111  0.105263  0.156297  0.555270   
...                       ...  ...       ...       ...       ...       ...   
2017-11-16 19:00:00  0.750000  0.2  0.333333  0.2105

In [51]:
# At last, a final check before exporting

if LOG:
    if years == [2017, 2018, 2020, 2021, 2022, 2023]:
        # Define the years for unsplitted and splitted data
        years_unsplitted = [2017, 2018, 2020]
        years_splitted = [2021, 2022]

        # First, check if u-dataframes of unsplitted years have the same shape
        u_data_unsplitted = [concat_frames_dict_u[year]['train'] for year in years_unsplitted]
        assert_equal_shape(u_data_unsplitted, True, True, 'Shape of u-dataframes of unsplitted years')

        # Second, check if y-dataframes of unsplitted years have the same shape
        y_data_unsplitted = [concat_frames_dict_y[year]['train'] for year in years_unsplitted]
        assert_equal_shape(y_data_unsplitted, True, True, 'Shape of y-dataframes of unsplitted years')

        # Third, check if validation/test u-dataframes of splitted years have the same shape
        u_data_splitted = [concat_frames_dict_u[year][split] for year in years_splitted for split in ['val', 'test']]
        assert_equal_shape(u_data_splitted, True, True, 'Shape of u-dataframes of splitted years')

        # Fourth, check if validation/test y-dataframes of splitted years have the same shape
        y_data_splitted = [concat_frames_dict_y[year][split] for year in years_splitted for split in ['val', 'test']]
        assert_equal_shape(y_data_splitted, True, True, 'Shape of y-dataframes of splitted years')

        print('(7/8): All data concatenations successful for years 2017, 2018, 2020, 2021, 2022')
    elif len(years) == 1:
        # print the shape of the concat frames for that year, for all train, val and test
        for split in splits:
            print(f"Shape of concatenated frames for {split} {years[0]}:")
            print(concat_frames_dict_u[years[0]][split].shape)
            print(concat_frames_dict_y[years[0]][split].shape)
    else:
        print("Error: Data concatenation not defined for the specific years")


Shape of concatenated frames for train 2017:
(2592, 9)
(2592, 1)
Shape of concatenated frames for val 2017:
(528, 9)
(528, 1)
Shape of concatenated frames for test 2017:
(528, 9)
(528, 1)


In [52]:
# Save the dataframes to data_combined/ folder. The windowing will be performed
# by a PyTorch Dataset class in the model scripts.

# Loop through each year and split to save the concatenated dataframes for Tuindorp
for year in years:
    for split in splits:
        try:
            df = concat_frames_dict_u[year][split]
            df.to_csv(f"../data/data_combined/{split}_{year}_combined_u.csv",
                      index=True, sep=';', decimal='.', encoding='utf-8')
            print(f"Saved {split}_{year}_combined_u.csv")
        except KeyError:
            print(f"Warning: {split}_{year}_combined_u does not exist.")

# Loop through each year and split to save the concatenated dataframes for Breukelen
for year in years:
    for split in splits:
        try:
            df = concat_frames_dict_y[year][split]
            df.to_csv(f"../data/data_combined/{split}_{year}_combined_y.csv",
                      index=True, sep=';', decimal='.', encoding='utf-8')
            print(f"Saved {split}_{year}_combined_y.csv")
        except KeyError:
            print(f"Warning: {split}_{year}_combined_y does not exist.")

Saved train_2017_combined_u.csv
Saved val_2017_combined_u.csv
Saved test_2017_combined_u.csv
Saved train_2017_combined_y.csv
Saved val_2017_combined_y.csv
Saved test_2017_combined_y.csv


In [53]:
if LOG:
    print('(8/8): Data exported successfully')
    print('Data preparation finished')

(8/8): Data exported successfully
Data preparation finished
